In [42]:
import general_robotics_toolbox as grt
from Arm_Lib import Arm_Device
import numpy as np
import time

#GLOBALS
NUM_JOINTS = 6
MIN_ANGLES = np.deg2rad(np.array([0,0,0,0,0,0]))
MAX_ANGLES = np.deg2rad(np.array([180,180,180,180,270,180]))
ZERO_CONFIG = np.deg2rad(np.array([0,0,0,0,0,0]))
in_2_m = 0.0254

In [43]:
def dofbot_robot():
    """REturns an approximate Robot instance for a DOFBOT robot"""
    x = np.array([1,0,0])
    y = np.array([0,1,0])
    z = np.array([0,0,1])
    a = np.array([0,0,0])
    
    H = np.array([z,y,y,y,z,y]).T
    P = np.array([a,4.5*z,3.5*z,3.5*z,3.0*z,a,4.0*z]).T*in_2_m
    JTs = [0,0,0,0,0,0]
    return grt.Robot(H,P,JTs,MIN_ANGLES,MAX_ANGLES)

In [44]:
def main():
    MEFA_BOT = dofbot_robot()
    pose = grt.fwdkin(MEFA_BOT,np.zeros(6))
    print(pose)
    
    

In [48]:
if __name__ == "__main__":
    Arm = Arm_Device()
    print("--STARTING--")
    time.sleep(.2)
    main()
    Arm.Arm_serial_servo_write(6,90,500)
    print("--END--")
    

--STARTING--
R = [[1. 0. 0.]
     [0. 1. 0.]
     [0. 0. 1.]]
p = [0.     0.     0.4699]


Arm_serial_servo_write I2C error
--END--
